This is the main script for the training and testing of the hybrid deepCNN-LSTM model,
used for the classification of abnormal EEG activity.

## Import Packages

In [ ]:
import import_ipynb
import numpy as np
from PreProcessing import import_tuh_abnormal, filter_only_adults, create_ch_mapping
from DataTransform import data_transform, train_valid_test_split, get_parameters_for_model
from TrainingValidationTesting_HybridBCELoss import train_model, test_model
import torch
from braindecode.preprocessing import preprocess, Preprocessor, scale as multiply
from Models import create_model
from braindecode.util import set_random_seeds
from torch import optim
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

# # Importing TUH Abnormal dataset

## Import TUH Abnormal Dataset

In [ ]:

TUH_PATH = 'TUH EEG Abnormal dataset/Abnormal/' # specify the path to the TUH Abnormal dataset
N_JOBS = 64  # specify the number of jobs for loading and windowing

# Parameters to be defined for the preprocessing pipeline
TMIN = 1 * 60
TMAX = 2.5 * 60 
SFREQ = 100
CLIPPING = 800

tuh_abnormal = import_tuh_abnormal(TUH_PATH, N_JOBS)

In [ ]:
# Filter for adults

tuh_abnormal_adults = filter_only_adults(tuh_abnormal)


In [ ]:
# Create short channel names and mapping    
short_ch_names, ch_mapping = create_ch_mapping()

def select_by_channels(ds, short_ch_names, ch_mapping):
    # Function for selecting channels defined in the input.
        
    split_ids = []
    for i, d in enumerate(ds.datasets):
        ref = 'ar' if d.raw.ch_names[0].endswith('-REF') else 'le'
        # these are the channels we are looking for
        seta = set(ch_mapping[ref].keys())
        # these are the channels of the recoding
        setb = set(d.raw.ch_names)
        # if recording contains all channels we are looking for, include it
        if seta.issubset(setb):
            split_ids.append(i)
    return ds.split(split_ids)['0']


def custom_crop(raw, tmin=0.0, tmax=None, include_tmax=True):       
    # Crop recordings to tmin – tmax. can be incomplete if recording has lower duration than tmax
    
    tmax = min((raw.n_times - 1) / raw.info['sfreq'], tmax)
    raw.crop(tmin=tmin, tmax=tmax, include_tmax=include_tmax)
    
    
def custom_rename_channels(raw, mapping):    
    # Rename channels which are dependent on referencing:
    # le: EEG 01-LE, ar: EEG 01-REF
    
    reference = raw.ch_names[0].split('-')[-1].lower()
    assert reference in ['le', 'ref'], 'unexpected referencing'
    reference = 'le' if reference == 'le' else 'ar'
    raw.rename_channels(mapping[reference])

# Preprocessing pipeline
preprocessors = [
        Preprocessor(custom_crop, tmin=TMIN, tmax=TMAX, include_tmax=True,
                     apply_on_array=False),
        Preprocessor('set_eeg_reference', ref_channels='average', ch_type='eeg'), # mne Raw class function
        Preprocessor(custom_rename_channels, mapping=ch_mapping, # rename channels to short channel names
                     apply_on_array=False), #
        Preprocessor('pick_channels', ch_names=short_ch_names, ordered=True), # mne Raw class function
        Preprocessor(multiply, factor=1e6, apply_on_array=True), # scaling signals to microvolt
        Preprocessor(np.clip, a_min=-CLIPPING, a_max=CLIPPING, apply_on_array=True), # clip outlier values to +/- 800 micro volts
        Preprocessor('resample', sfreq=SFREQ), # mne Raw class function
    ]


tuh_preproc = preprocess(
        concat_ds=tuh_abnormal, #input either tuh_abnormal or tuh_abnormal_adults
        preprocessors=preprocessors,
        n_jobs=N_JOBS,
        save_dir=None
    )

## Transform Data and Instantiate Model

In [ ]:
# Execute train-validation-test split

TRAIN_SIZE=0.9 # Parameter for defining the ratio of training set (remainder defines the size of the validation set)

tuh_train, tuh_val, tuh_test = train_valid_test_split(tuh_preproc, train_size=TRAIN_SIZE)

In [ ]:
cuda = torch.cuda.is_available()  # check if GPU is available, if True, use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True

seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

# Required parameters for model creation
INPUT_WIN_SAMPLES = 6000
IN_CHANS, INPUT_SIZE_SAMPLES = get_parameters_for_model(tuh_train)
N_CLASSES = 2
MODEL_NAME = "hybrid" # it can either be shallow, deep, or hybrid
FINAL_CONV_LENGTH = 1

model = create_model(MODEL_NAME, IN_CHANS, N_CLASSES, INPUT_WIN_SAMPLES, FINAL_CONV_LENGTH)


# If available, use multiple GPUs for training and testing
if cuda:
    device = "cuda:0"
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model, device_ids = [ 0, 1])
model.to(device)
    
    

In [ ]:
# Execute DataLoader generating function

BATCH_SIZE=64

train_loader, val_loader, test_loader = data_transform(tuh_train, tuh_val, tuh_test, model, N_JOBS, train_size=TRAIN_SIZE, batch_size=BATCH_SIZE, input_window_samples=INPUT_WIN_SAMPLES, in_chans = IN_CHANS)

In [ ]:
# Hyperparameters required for the training of the model

lr =  0.01
optimizer = optim.Adam(params=model.parameters(), lr=lr)
criterion = torch.nn.BCELoss()
n_epochs = 1 
patience = 1 #ES

In [ ]:
# Model Training

trained_model, train_losses, train_accs, valid_losses, valid_accs = train_model(model, device, criterion, optimizer, train_loader, val_loader, n_epochs, patience)
trained_model.to(device)

In [ ]:
# Save the trained model

PATH = './modelname.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
# Model testing

x_test, y_test, y_pred, accuracy = test_model(device, trained_model, test_loader)

## Evaluation Metrics

In [ ]:
# Reshape tensors

x_test = x_test.flatten().cpu()
y_test = y_test.flatten().cpu()
y_pred = y_pred.flatten().cpu()

In [ ]:
# Training loss and accuracy per epoch

fig,ax = plt.subplots()
ax.plot(train_losses, marker="o", color="orangered", label="Training loss")
ax.set_xlabel("Number of epochs")
ax.set_ylabel("Loss", color="orangered")

# twin object for two different y-axis on the sample plot
ax2=ax.twinx()
ax2.plot(train_accs, marker="o", color="cornflowerblue", label="Training accuracy")
ax2.set_ylabel("Accuracy", color="cornflowerblue")

plt.title("Training loss and accuracy per epoch", fontsize=14)
plt.show()


In [ ]:
# Learning curve with training and validation loss

ax = plt.axes()
plt.plot(train_losses, marker=".", color="cornflowerblue", label='Training loss')
plt.plot(valid_losses, marker=".", color="mediumseagreen", label='Validation loss')
plt.title('Training and Validation Losses', fontsize=14)
plt.xlabel('Number of epochs', fontsize=14)
plt.legend(frameon=False, fontsize=14)
ax.tick_params(axis='both', labelsize=13)
plt.show()

In [ ]:
''' 
The ROC Curve plots two parameters: the True Positive Rate (TPR), a synonim to recall,
and the False Positive Rate (FPR). 
The plot also shows the ROC AUC score.
By default, the class considered as the positive (It means Abnormal in our case).

'''

RocCurveDisplay.from_predictions(y_test, y_pred, pos_label=None)
plt.title("ROC Curve", fontsize=14)
plt.show()


In [ ]:
# In[90]: Confusion matrix and Classification Report

conf_matrix = confusion_matrix(y_test, y_pred)
cl_report = classification_report(y_test, y_pred)

print(cl_report)

In [ ]:
# Confusion matrix plot

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                conf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     conf_matrix.flatten()/np.sum(conf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)


sns.heatmap(conf_matrix, annot=labels, fmt='', cmap='YlGn')
sns.set(font_scale=1.6)
ax.set_title('Confusion Matrix', fontsize=16)
ax.set_xlabel('Predicted labels', fontsize=14)
ax.set_ylabel('True labels', fontsize=14)
ax.tick_params(axis='both', labelsize=13)
plt.show()